In [1]:
import torch
from IPython import display
from d2l import torch as d2l

In [2]:
batch_size = 256
mnist_train, mnist_test = d2l.load_data_fashion_mnist(batch_size)

/home/qiurong/venv/d2l-zh/lib/python3.6/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# 初始化参数

正太分布初始化权重W；

偏置初始化为0;

In [3]:
num_inputs = 784
num_outputs = 10
W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

# 定义softmax回归函数

1. 求幂
2. 求和
3. 归一化

In [13]:
def softmax(X):
    """对矩阵X做softmax运算"""
    X_exp = torch.exp(X)
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition

In [14]:
X = torch.normal(0, 1, size=(2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.3800, 0.1014, 0.1882, 0.0651, 0.2654],
         [0.2341, 0.0292, 0.3047, 0.2594, 0.1725]]),
 tensor([1.0000, 1.0000]))

In [31]:
torch.exp(torch.tensor([50]))

tensor([5.1847e+21])

# 定义模型

softmax回归模型

In [15]:
def net(X):
    """定义图像的softmax回归模型"""
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

# 定义损失函数

交叉熵损失函数

In [21]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

# 分类准确率

In [23]:
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and len(y_hat.shape[1]) > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [26]:
class Accumulator: #@save
    """在`n`个变量上累加。 """
    def __init__(self, n):
        self.data = [0.0] * n
        
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
        
    def reset(self):
        self.data = [0.0] * len(self.data)
        
    def __getitem__(self, idx):
        return self.data[idx]

In [27]:
def evaluate_accuracy(net, data_iter):
    """计算在数据集上的准确率"""
    if isinstance(net, torch.nn.Module):
        net.eval()
    metric = Accumulator(2)
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

# 训练

In [28]:
def train_epoch_ch3(net, train_iter, loss, updater):
    """训练模型的一个迭代周期"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    # 批量迭代计算梯度并更新参数
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            updater.step()
            metric.add(float(l) * len(y),
                      accuracy(y_hat, y),
                      y.size().numel())
        else:
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(l.sum()),
                      accuracy(y_hat, y),
                      y.numel())
        # 返回训练损失和准确度
        return metric[0]/metric[2], metric[1]/metric[2]

SyntaxError: unexpected EOF while parsing (<ipython-input-28-e2d64d393171>, line 1)